In [1]:
import os
import pandas as pd
import pickle
import ast
import sys
import re

In [2]:
OUT_DIR="/media/SSD/paper_data/"

In [8]:
sensor_tag="all"
seeds = [0]#, 1, 2, 3, 4]
preprocessings = ["none"]#, "CWT-H", "CWT-V"]
models = ['resnet']#["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
averagings = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
i=10000
K_FOLD =1

columns = ['dataset', 'averaging', 'model', 'preprocessing', 'seed', 'accuracy']
accuracies = pd.DataFrame(columns=columns)

results_idx = 0

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                for average in averagings:

                    model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                    acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_a"+str(average)+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_"+str(K_FOLD)+"fold/out/record/LSTM+CNN_record.txt"

                    # Check if job was completed
                    if(os.path.exists(acc_file)!=True):
                        print("Results for "+acc_file+" not yet ready!")
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, average, model, preprocessing, seed, "-"]
                        results_idx = results_idx+1
                        continue

                    # Check if data was stored safetly
                    num_lines = sum(1 for line in open(acc_file))
                    if(num_lines < 1):
                        print("Missing data for "+acc_file+" not yet ready!")
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, average, model, preprocessing, seed, "-"]
                        results_idx = results_idx+1
                        continue
                    #print(out_path)

                    file1 = open(acc_file, 'r')
                    Lines = file1.readlines()

                    for count, line in enumerate(Lines):
                        #print("Line{}: {}".format(count, line.strip()))
                        if(count==4):
                            accuracy = re.findall("\d+\.\d+", line.strip())[0]
                            #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                            #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, average, model, preprocessing, seed, float(accuracy)]
                            results_idx = results_idx+1

                    file1.close()

In [9]:
accuracies.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_averaging.csv", index=False)